In [ ]:
!wget train_reviews.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
!wget train_aspects.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_aspects.txt
!wget train_split_aspects.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_split_aspects.txt

--2022-12-27 18:48:43--  http://train_reviews.txt/
Resolving train_reviews.txt (train_reviews.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘train_reviews.txt’
--2022-12-27 18:48:43--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/train_reviews.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 446118 (436K) [text/plain]
Saving to: ‘train_reviews.txt’

train_reviews.txt   100%[===================>] 435.66K  --.-KB/s    in 0.04s   

2022-12-27 18:48:43 (11.0 MB/s) - ‘train_reviews.txt’ saved [446118/446118]

FINISHED --2022-12-27 18:48:43--
Total wall clock time: 0.4s
Downloaded: 1 files, 436K in 0.04s (11.0 MB/s)
--2022-12-27 18:48:43--  http://train_aspects.txt/
Resolving train_aspects.txt 

In [ ]:
import pandas as pd
train_reviews = pd.read_csv('train_reviews.txt', sep='\t', names=['review_id', 'text'], header=None)
train_aspects = pd.read_csv('train_split_aspects.txt', sep='\t', names=['review_id', 'category', 'span', 'span_start', 'span_end', 'sentiment'], header=None)

In [ ]:
!python3 -m spacy download ru_core_news_md

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2022-12-27 18:48:59.518584: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 3.1 MB/s 
     |████████████████████████████████| 55 kB 2.0 MB/s 
     |████████████████████████████████| 8.2 MB 9.5 MB/s 
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13723 sha256=7fb8372dfa011c821ecd9c434ec5af082723a0c347c239e78c460c77cf71ff6c
  Stored in directory: /root/.cache/pip/wheels/56/ea/58/ead137b087d9e326852a851351d1debf4ada529b6ac0ec4e8c
Successfully built docopt
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


#Предобработка

### Получаем зависимости для всех токенов и индексы для сопоставления с тем, что мы уже имеем

Берем наиболее интересные части речи в children

In [ ]:
import spacy
import re
from tqdm.notebook import tqdm
tqdm.pandas()

def get_dependencies(text, review_id):
  all_tokens = []
  doc = nlp(text)
  for token in doc:
    all_tokens.append([review_id, token.text, token.pos_, token.dep_, [child for child in token.children if child.pos_ in ['ADJ', 'NOUN', 'VERB', 'PART']], token.head.text, token.idx, token.idx+len(token.text)]) 
  global result_spacy
  result_spacy.append(all_tokens)

result_spacy = []
nlp = spacy.load("ru_core_news_md")
train_reviews.progress_apply(lambda x: get_dependencies(x['text'], x['review_id']), axis=1)

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


  0%|          | 0/284 [00:00<?, ?it/s]

0      None
1      None
2      None
3      None
4      None
       ... 
279    None
280    None
281    None
282    None
283    None
Length: 284, dtype: object

Сопоставляем по индексам с нашим исходником, собираем импровизированный чанк



In [ ]:
def find_for_train(span, start, end, review_id):
  s = span
  for i in range(0, len(result_spacy)):
      for j in range(0, len(result_spacy[i])):
        if review_id == result_spacy[i][j][0] and span == result_spacy[i][j][1] and start == result_spacy[i][j][6] and end == result_spacy[i][j][7]:
          s = result_spacy[i][j][5] + ' ' + result_spacy[i][j][1]
          if type(result_spacy[i][j][4]) == list:
            for x in result_spacy[i][j][4]:
              s = s + ' ' + x.text
          if type(result_spacy[i][j][4]) == str:
              s = s + ' ' + result_spacy[i][j][4].text

  return s
      

train_aspects['chunk'] = train_aspects.progress_apply(lambda x: find_for_train(x['span'], x['span_start'], x['span_end'], x['review_id']), axis=1)

  0%|          | 0/3573 [00:00<?, ?it/s]

# Обучение

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 5.0 MB/s 
     |████████████████████████████████| 7.6 MB 58.2 MB/s 
     |████████████████████████████████| 182 kB 72.8 MB/s 


Берем несколько моделей для экспериментов

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
import lightgbm as lgb
from sklearn.metrics import classification_report

In [ ]:
!wget dev_aspects.txt https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
dev_aspects = pd.read_csv('dev_aspects.txt', sep='\t', names=['review_id', 'category', 'span', 'span_start', 'span_end', 'sentiment'], header=None)

--2022-12-27 18:50:50--  http://dev_aspects.txt/
Resolving dev_aspects.txt (dev_aspects.txt)... failed: Name or service not known.
wget: unable to resolve host address ‘dev_aspects.txt’
--2022-12-27 18:50:50--  https://raw.githubusercontent.com/named-entity/hse-nlp/master/4th_year/Project/dev_aspects.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 57508 (56K) [text/plain]
Saving to: ‘dev_aspects.txt’

dev_aspects.txt     100%[===================>]  56.16K  --.-KB/s    in 0.01s   

2022-12-27 18:50:50 (4.80 MB/s) - ‘dev_aspects.txt’ saved [57508/57508]

FINISHED --2022-12-27 18:50:50--
Total wall clock time: 0.2s
Downloaded: 1 files, 56K in 0.01s (4.80 MB/s)


### Получаем вектора LaBSE

In [ ]:
def get_embeddings_LaBSE(texts, tokenizer_LaBSE, model_LaBSE):
  tokenizer_LaBSE = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
  model_LaBSE = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")
  encoded_input = tokenizer_LaBSE(texts, padding=True, truncation=True, return_tensors='pt')
  with torch.no_grad():
      model_output = model_LaBSE(**encoded_input)
  embeddings_LaBSE = model_output.pooler_output
  embeddings_LaBSE = torch.nn.functional.normalize(embeddings_LaBSE)
  return embeddings_LaBSE


tokenizer_LaBSE = AutoTokenizer.from_pretrained("cointegrated/LaBSE-en-ru")
model_LaBSE = AutoModel.from_pretrained("cointegrated/LaBSE-en-ru")

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/806 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/521k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/516M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
embeddings_LaBSE = get_embeddings_LaBSE(list(train_aspects['span']), tokenizer_LaBSE, model_LaBSE)
dev_embeddings_LaBSE = get_embeddings_LaBSE(list(dev_aspects['span']), tokenizer_LaBSE, model_LaBSE)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.se

### LaBSE + LogisticRegression

In [ ]:
clf = LogisticRegression(random_state = 200).fit(embeddings_LaBSE, train_aspects['sentiment'])
predict_labels = clf.predict(dev_embeddings_LaBSE)
print(classification_report(list(dev_aspects['sentiment']), predict_labels))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.69      0.06      0.10       160
     neutral       0.47      0.17      0.25       216
    positive       0.69      0.96      0.81       795

    accuracy                           0.68      1190
   macro avg       0.47      0.30      0.29      1190
weighted avg       0.64      0.68      0.60      1190



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### LaBSE + XGBClassifier

In [ ]:
model_xgb = XGBClassifier(max_depth=6, n_estimators=400)
pipeline = Pipeline(steps=[('m', model_xgb)])
pipeline.fit(embeddings_LaBSE, list(train_aspects['sentiment']))
y_pred = pipeline.predict(dev_embeddings_LaBSE)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.10      0.05      0.07        19
    negative       0.36      0.13      0.19       160
     neutral       0.50      0.33      0.40       216
    positive       0.72      0.89      0.80       795

    accuracy                           0.67      1190
   macro avg       0.42      0.35      0.36      1190
weighted avg       0.62      0.67      0.63      1190



### LaBSE + LGBMClassifier

In [ ]:
model_lgb = lgb.LGBMClassifier(max_depth=9, learning_rate=0.0317, num_class=4, n_estimators=100)
pipeline = Pipeline(steps=[('m', model_lgb)])
pipeline.fit(embeddings_LaBSE, list(train_aspects['sentiment']))
y_pred = pipeline.predict(dev_embeddings_LaBSE)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.42      0.09      0.15       160
     neutral       0.53      0.29      0.38       216
    positive       0.72      0.93      0.81       795

    accuracy                           0.69      1190
   macro avg       0.42      0.33      0.33      1190
weighted avg       0.63      0.69      0.63      1190



### Получаем вектора RuBert-tiny

In [ ]:
tokenizer_rubert = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
model_rubert = AutoModel.from_pretrained("cointegrated/rubert-tiny")

def embed_bert_cls(text, model, tokenizer):
    t = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**{k: v.to(model.device) for k, v in t.items()})
    embeddings = model_output.last_hidden_state[:, 0, :]
    embeddings = torch.nn.functional.normalize(embeddings)
    return embeddings

Downloading:   0%|          | 0.00/341 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/632 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/241k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/468k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/47.7M [00:00<?, ?B/s]

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
embeddings_bert = embed_bert_cls(list(train_aspects['span']), model_rubert, tokenizer_rubert)
embeddings_bert_dev = embed_bert_cls(list(dev_aspects['span']), model_rubert, tokenizer_rubert)

### rubert-tiny + XGBClassifier

In [ ]:
model_xgb_2 = XGBClassifier(max_depth=3, n_estimators=50)
pipeline = Pipeline(steps=[('m', model_xgb_2)])
pipeline.fit(embeddings_bert, list(train_aspects['sentiment']))
y_pred = pipeline.predict(embeddings_bert_dev)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.50      0.02      0.04       160
     neutral       0.69      0.16      0.26       216
    positive       0.69      0.98      0.81       795

    accuracy                           0.69      1190
   macro avg       0.47      0.29      0.28      1190
weighted avg       0.65      0.69      0.59      1190



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### rubert-tiny + LGBMClassifier

In [ ]:
model = lgb.LGBMClassifier(max_depth=9, learning_rate=0.0317, num_class=4, n_estimators=50)
pipeline = Pipeline(steps=[('m', model)])
pipeline.fit(embeddings_bert, list(train_aspects['sentiment']))
y_pred = pipeline.predict(embeddings_bert_dev)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.10      0.05      0.07        19
    negative       0.45      0.09      0.15       160
     neutral       0.52      0.24      0.33       216
    positive       0.70      0.93      0.80       795

    accuracy                           0.68      1190
   macro avg       0.44      0.33      0.34      1190
weighted avg       0.63      0.68      0.62      1190



### Получаем вектора Sbert_large

In [ ]:
tokenizer_sbert = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")
model_sbert = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

embeddings_sbert = embed_bert_cls(list(train_aspects['span']), model_sbert, tokenizer_sbert)
embeddings_sbert_dev = embed_bert_cls(list(dev_aspects['span']), model_sbert, tokenizer_sbert)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


### Sbert_large + XGBClassifier

In [ ]:
model = XGBClassifier()
pipeline = Pipeline(steps=[('m', model)])
pipeline.fit(embeddings_sbert, list(train_aspects['sentiment']))
y_pred = pipeline.predict(embeddings_sbert_dev)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.49      0.12      0.20       160
     neutral       0.50      0.29      0.37       216
    positive       0.72      0.93      0.81       795

    accuracy                           0.69      1190
   macro avg       0.43      0.33      0.34      1190
weighted avg       0.64      0.69      0.63      1190



### Sbert_large + LBMClassifier

In [ ]:
model = lgb.LGBMClassifier()
pipeline = Pipeline(steps=[('m', model)])
pipeline.fit(embeddings_sbert, list(train_aspects['sentiment']))
y_pred = pipeline.predict(embeddings_sbert_dev)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.42      0.14      0.21       160
     neutral       0.48      0.34      0.40       216
    positive       0.72      0.89      0.80       795

    accuracy                           0.67      1190
   macro avg       0.41      0.34      0.35      1190
weighted avg       0.63      0.67      0.63      1190



In [ ]:
dev_aspects['chunk'] = dev_aspects.progress_apply(lambda x: find_for_train(x['span'], x['span_start'], x['span_end'], x['review_id']), axis=1)

  0%|          | 0/1190 [00:00<?, ?it/s]

### Получаем вектора для наших собранных чанков

Особого различия в зависимости от модели для векторизации не замечено, возьмем LaBSE

In [ ]:
embeddings_labse_chunk = get_embeddings_LaBSE(list(train_aspects['chunk']), model_LaBSE, tokenizer_LaBSE)
embeddings_labse_dev_chunk = get_embeddings_LaBSE(list(dev_aspects['chunk']), model_LaBSE, tokenizer_LaBSE)

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.se

In [ ]:
model_xgb = XGBClassifier(max_depth=3, n_estimators=100)
model_xgb.fit(embeddings_labse_chunk, list(train_aspects['sentiment']))
y_pred = model_xgb.predict(embeddings_labse_dev_chunk)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.72      0.17      0.28       160
     neutral       0.53      0.28      0.36       216
    positive       0.71      0.93      0.81       795

    accuracy                           0.70      1190
   macro avg       0.49      0.35      0.36      1190
weighted avg       0.67      0.70      0.64      1190



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Accuracy наконец-то изменилось в большую сторону, сохраняем

In [ ]:
import pickle
pkl_filename = "xgb_labse_chunk.pkl" 
with open(pkl_filename, 'wb') as file: 
  pickle.dump(model_xgb, file)

Тут тоже поднялось, но все еще меньше, чем предыдущее

In [ ]:
model_lgb_chunk = lgb.LGBMClassifier(max_depth=5, learning_rate=0.0317, num_class=4, n_estimators=100)
pipeline = Pipeline(steps=[('m', model_lgb_chunk)])
pipeline.fit(embeddings_labse_chunk, list(train_aspects['sentiment']))
y_pred = pipeline.predict(embeddings_labse_dev_chunk)
print(classification_report(list(dev_aspects['sentiment']), y_pred))

              precision    recall  f1-score   support

        both       0.00      0.00      0.00        19
    negative       0.89      0.11      0.19       160
     neutral       0.54      0.20      0.29       216
    positive       0.70      0.96      0.81       795

    accuracy                           0.69      1190
   macro avg       0.53      0.32      0.32      1190
weighted avg       0.69      0.69      0.62      1190



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
